# RO System Simulation with MCAS - Simplified Version

This simplified MCAS notebook template uses fixed pressure drops per stage to avoid
the need to specify length and width of the membrane. Simply provide the total area per stage.

In [ ]:
# Parameters cell - will be replaced by papermill
project_root = "/path/to/project"  # Will be replaced by papermill
configuration = {}
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
membrane_properties = None
optimize_pumps = False
# Ion composition in mg/L (optional, if not provided will use typical composition)
feed_ion_composition = None
# Initialization strategy: "sequential", "block_triangular", "custom_guess", "relaxation"
initialization_strategy = "sequential"

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from pyomo.environ import *
from pyomo.network import Arc
import sys
import os

# Add parent directory to path for utils imports using project_root parameter
sys.path.insert(0, project_root)

# Import our MCAS builder
from utils.mcas_builder import (
    build_mcas_property_configuration,
    check_electroneutrality,
    get_total_dissolved_solids,
    calculate_ionic_strength
)

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType
)
from watertap.unit_models.pressure_changer import Pump
from watertap.property_models.multicomp_aq_sol_prop_pack import MCASParameterBlock
from watertap.core import ModuleType

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.models.unit_models import Feed, Product

import warnings
warnings.filterwarnings('ignore')

# Results storage
results = {}

## Define Feed Ion Composition

In [ ]:
# Define typical ion compositions for different water types
TYPICAL_COMPOSITIONS = {
    "brackish": {
        "Na+": 1200,
        "Ca2+": 120,
        "Mg2+": 60,
        "K+": 20,
        "Cl-": 2100,
        "SO4-2": 200,
        "HCO3-": 150,
        "SiO3-2": 10
    },
    "seawater": {
        "Na+": 10800,
        "Ca2+": 420,
        "Mg2+": 1300,
        "K+": 400,
        "Sr2+": 8,
        "Cl-": 19400,
        "SO4-2": 2700,
        "HCO3-": 140,
        "Br-": 70,
        "F-": 1.3
    }
}

# Use provided composition or typical based on salinity
if feed_ion_composition is None:
    # Scale typical composition to match target salinity
    typical = TYPICAL_COMPOSITIONS[membrane_type]
    typical_tds = sum(typical.values())
    scale_factor = feed_salinity_ppm / typical_tds
    
    feed_ion_composition = {
        ion: conc * scale_factor 
        for ion, conc in typical.items()
    }
else:
    # Parse JSON string if provided
    import json
    if isinstance(feed_ion_composition, str):
        feed_ion_composition = json.loads(feed_ion_composition)

# Check and report composition
print(f"Feed water composition (mg/L):")
for ion, conc in sorted(feed_ion_composition.items()):
    print(f"  {ion:8s}: {conc:8.1f}")

actual_tds = get_total_dissolved_solids(feed_ion_composition)
print(f"\nTotal TDS: {actual_tds:.0f} mg/L")

# Check electroneutrality
is_neutral, imbalance = check_electroneutrality(feed_ion_composition)
if not is_neutral:
    print(f"\nWarning: Charge imbalance of {imbalance:.1%}")

# Calculate ionic strength
ionic_strength = calculate_ionic_strength(feed_ion_composition)
print(f"Ionic strength: {ionic_strength:.3f} mol/L")

## Build MCAS Property Configuration

In [ ]:
# Build MCAS property configuration
mcas_config = build_mcas_property_configuration(
    feed_composition=feed_ion_composition,
    include_scaling_ions=True,
    include_ph_species=True
)

print("MCAS Configuration created with components:")
print(f"  Solute list: {', '.join(mcas_config['solute_list'])}")
print(f"  Activity model: {mcas_config['activity_coefficient_model']}")
print(f"  Scaling ions tracked: {len(mcas_config.get('scaling_ions', {}))} types")

## Build Simplified WaterTAP Model with MCAS

In [ ]:
def build_ro_model_mcas_simple(config_data, mcas_config, feed_temperature_c, membrane_type):
    """
    Build simplified WaterTAP RO model with MCAS property package.
    Uses fixed pressure drops to avoid membrane geometry specification.
    """
    # Create concrete model
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Import MaterialFlowBasis for mass flow compatibility with RO
    from watertap.property_models.multicomp_aq_sol_prop_pack import MaterialFlowBasis
    
    # Filter MCAS config to only include parameters MCASParameterBlock accepts
    mcas_params = {
        'solute_list': mcas_config['solute_list'],
        'mw_data': mcas_config['mw_data'],
        'material_flow_basis': MaterialFlowBasis.mass,  # Use mass flow for RO compatibility
    }
    
    # Add optional parameters if they exist
    optional_params = [
        'stokes_radius_data', 'diffusivity_data', 'molar_volume_data',
        'elec_mobility_data', 'trans_num_data', 'equiv_conductivity_phase_data',
        'charge', 'ignore_neutral_charge', 'activity_coefficient_model',
        'density_calculation', 'diffus_calculation', 'elec_mobility_calculation',
        'trans_num_calculation'
    ]
    
    for param in optional_params:
        if param in mcas_config:
            mcas_params[param] = mcas_config[param]
    
    # Create MCAS property package with filtered parameters
    m.fs.properties = MCASParameterBlock(**mcas_params)
    
    # Feed conditions
    feed_flow_m3_s = config_data['feed_flow_m3h'] / 3600  # Convert to m³/s
    
    # Create feed block
    m.fs.feed = Feed(property_package=m.fs.properties)
    
    # Build stages - use proper attribute assignment for pumps and RO units
    n_stages = config_data['stage_count']
    
    # First, create all units (without setting parameters)
    for i in range(1, n_stages + 1):
        # Create pump for stage using setattr to ensure proper parent assignment
        setattr(m.fs, f"pump{i}", Pump(property_package=m.fs.properties))
        
        # Create RO stage with simplified configuration:
        # - fixed_per_stage pressure drop
        # - none concentration polarization (simpler, avoids channel geometry)
        # - none mass transfer coefficient
        # - Add spiral wound module type
        setattr(m.fs, f"ro_stage{i}", ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            concentration_polarization_type=ConcentrationPolarizationType.none,
            mass_transfer_coefficient=MassTransferCoefficient.none,
            pressure_change_type=PressureChangeType.fixed_per_stage,
            module_type=ModuleType.spiral_wound  # Add spiral wound module type
        ))
        
        # Create permeate product for each stage
        setattr(m.fs, f"stage_product{i}", Product(property_package=m.fs.properties))
    
    # Build connectivity
    # Feed to first pump
    m.fs.feed_to_pump1 = Arc(
        source=m.fs.feed.outlet,
        destination=m.fs.pump1.inlet
    )
    
    # First pump to first RO
    m.fs.pump1_to_ro1 = Arc(
        source=m.fs.pump1.outlet,
        destination=m.fs.ro_stage1.inlet
    )
    
    # Permeate from first RO to product
    m.fs.ro1_perm_to_prod = Arc(
        source=m.fs.ro_stage1.permeate,
        destination=m.fs.stage_product1.inlet
    )
    
    # Connect stages if multiple
    if n_stages > 1:
        for i in range(1, n_stages):
            # Concentrate of stage i to pump i+1
            arc_name = f"ro{i}_to_pump{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"ro_stage{i}").retentate,
                destination=getattr(m.fs, f"pump{i+1}").inlet
            ))
            
            # Pump i+1 to RO i+1
            arc_name = f"pump{i+1}_to_ro{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"pump{i+1}").outlet,
                destination=getattr(m.fs, f"ro_stage{i+1}").inlet
            ))
            
            # Permeate to product
            arc_name = f"ro{i+1}_perm_to_prod{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"ro_stage{i+1}").permeate,
                destination=getattr(m.fs, f"stage_product{i+1}").inlet
            ))
    
    # Final concentrate product
    m.fs.concentrate_product = Product(property_package=m.fs.properties)
    final_stage = n_stages
    m.fs.final_conc_arc = Arc(
        source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
        destination=m.fs.concentrate_product.inlet
    )
    
    # Apply arcs to expand the network
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    # NOW set membrane properties after model structure is built
    for i in range(1, n_stages + 1):
        stage_data = config_data['stages'][i-1]
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Membrane properties based on type
        if membrane_type == "seawater":
            ro.A_comp.fix(1.5e-12)  # m/s/Pa
            # Set B values only for solutes (not H2O)
            for comp in mcas_config['solute_list']:
                # Different rejection for different ions
                if comp in ['Na+', 'Cl-']:
                    ro.B_comp[0, comp].fix(1.0e-8)  # m/s
                elif comp in ['Ca2+', 'Mg2+', 'SO4-2']:
                    ro.B_comp[0, comp].fix(5.0e-9)  # m/s - higher rejection
                else:
                    ro.B_comp[0, comp].fix(8.0e-9)  # m/s
        else:  # brackish
            ro.A_comp.fix(4.2e-12)  # m/s/Pa
            for comp in mcas_config['solute_list']:
                if comp in ['Na+', 'Cl-']:
                    ro.B_comp[0, comp].fix(3.5e-8)  # m/s
                elif comp in ['Ca2+', 'Mg2+', 'SO4-2']:
                    ro.B_comp[0, comp].fix(1.5e-8)  # m/s
                else:
                    ro.B_comp[0, comp].fix(2.5e-8)  # m/s
        
        # For fixed_per_stage pressure change type with spiral wound modules:
        # While we still use the simplified approach with fixed pressure drop,
        # the spiral wound module type ensures proper mass transfer correlations
        # are used for the membrane calculations
        
        # Set membrane area directly (no need for length/width)
        ro.area.fix(stage_data['membrane_area_m2'])
        
        # Set pressure drop for the stage
        # Use reasonable defaults based on stage number
        if i == 1:
            ro.deltaP.fix(-0.5e5)  # -0.5 bar pressure drop
        elif i == 2:
            ro.deltaP.fix(-0.7e5)  # -0.7 bar pressure drop
        else:
            ro.deltaP.fix(-1.0e5)  # -1.0 bar pressure drop
        
        # Fix permeate pressure (typically atmospheric)
        ro.permeate.pressure.fix(101325)  # 1 atm
    
    # Set feed conditions
    feed_state = m.fs.feed.outlet
    
    # Temperature and pressure
    feed_state.temperature.fix(273.15 + feed_temperature_c)
    feed_state.pressure.fix(101325)  # 1 atm
    
    # Component flows based on ion composition (now using mass flow)
    ion_composition_mg_l = mcas_config['ion_composition_mg_l']
    total_ion_flow_kg_s = 0
    
    # Set ion flows (mass basis)
    for comp in mcas_config['solute_list']:
        # Ion flow
        conc_mg_l = ion_composition_mg_l[comp]
        ion_flow_kg_s = conc_mg_l * feed_flow_m3_s / 1000  # mg/L * m³/s / 1000 = kg/s
        feed_state.flow_mass_phase_comp[0, 'Liq', comp].fix(ion_flow_kg_s)
        total_ion_flow_kg_s += ion_flow_kg_s
    
    # Water flow (mass basis)
    water_flow_kg_s = feed_flow_m3_s * 1000 - total_ion_flow_kg_s  # Assume density ~1000 kg/m³
    feed_state.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(water_flow_kg_s)
    
    # Set pump efficiencies
    for i in range(1, n_stages + 1):
        getattr(m.fs, f"pump{i}").efficiency_pump.fix(0.8)
    
    return m

## Initialize Model with Robust Strategy

In [ ]:
def initialize_model_simple(m, config_data):
    """
    Initialize simplified model with sequential strategy.
    Pump outlet pressures are left as decision variables.
    """
    # Set scaling factors for better convergence
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1, index=('Liq', 'H2O'))
    # Use solute_set instead of solute_list
    for comp in m.fs.properties.solute_set:
        if comp in ['Na+', 'Cl-']:
            m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e3, index=('Liq', comp))
        else:
            m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e4, index=('Liq', comp))
    m.fs.properties.set_default_scaling('pressure', 1e-5)
    m.fs.properties.set_default_scaling('temperature', 1e-2)
    
    # Initialize feed
    m.fs.feed.initialize()
    
    # Set initial guesses for pump outlet pressures
    pressure_guesses = {
        1: 15e5,  # 15 bar for stage 1
        2: 20e5,  # 20 bar for stage 2
        3: 25e5   # 25 bar for stage 3
    }
    
    # Initialize stages sequentially
    for i in range(1, config_data['stage_count'] + 1):
        print(f"\nInitializing Stage {i}...")
        
        # Initialize pump
        pump = getattr(m.fs, f"pump{i}")
        
        # Propagate state from previous unit
        if i == 1:
            propagate_state(arc=m.fs.feed_to_pump1)
        else:
            arc_name = f"ro{i-1}_to_pump{i}"
            propagate_state(arc=getattr(m.fs, arc_name))
        
        # Set bounds and initial value for pump outlet pressure
        pump.outlet.pressure[0].setlb(5e5)   # 5 bar minimum
        pump.outlet.pressure[0].setub(80e5)  # 80 bar maximum
        pump.outlet.pressure[0].set_value(pressure_guesses.get(i, 20e5))
        
        # Get inlet conditions for pump
        inlet_flow_h2o = value(pump.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        inlet_temp = value(pump.inlet.temperature[0])
        
        # Build inlet flows dict
        inlet_flows = {('Liq', 'H2O'): inlet_flow_h2o}
        # Use solute_set instead of solute_list
        for comp in m.fs.properties.solute_set:
            inlet_flows[('Liq', comp)] = value(pump.inlet.flow_mass_phase_comp[0, 'Liq', comp])
        
        # Initialize pump with explicit outlet state to avoid pressure bounds issue
        pump.initialize(
            state_args={
                "pressure": pressure_guesses.get(i, 20e5),
                "temperature": inlet_temp,
                "flow_mass_phase_comp": inlet_flows
            }
        )
        
        # Initialize RO
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Propagate state from pump
        if i == 1:
            propagate_state(arc=m.fs.pump1_to_ro1)
        else:
            arc_name = f"pump{i}_to_ro{i}"
            propagate_state(arc=getattr(m.fs, arc_name))
        
        # Apply scaling to RO unit
        from idaes.core.util.scaling import calculate_scaling_factors
        calculate_scaling_factors(ro)
        
        # Initialize RO with relaxed tolerances
        ro.initialize(
            optarg={
                'tol': 1e-6,
                'constr_viol_tol': 1e-6,
                'nlp_scaling_method': 'user-scaling',
                'linear_solver': 'ma27'
            }
        )
        
        # Initialize stage product
        if i == 1:
            propagate_state(arc=m.fs.ro1_perm_to_prod)
        else:
            arc_name = f"ro{i}_perm_to_prod{i}"
            propagate_state(arc=getattr(m.fs, arc_name))
        
        getattr(m.fs, f"stage_product{i}").initialize()
    
    # Initialize final concentrate product
    propagate_state(arc=m.fs.final_conc_arc)
    m.fs.concentrate_product.initialize()
    
    print("\nInitialization complete.")

## Solve Model with Scaling

In [ ]:
def solve_with_scaling_simple(m, config_data, optimize_pumps=False):
    """
    Solve simplified model with appropriate scaling.
    Pump pressures are calculated to meet target recoveries.
    """
    # Apply scaling factors for mass flow basis
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e2, index=('Liq', 'H2O'))
    # Use solute_set instead of solute_list
    for comp in m.fs.properties.solute_set:
        if comp in ['Na+', 'Cl-']:
            m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e3, index=('Liq', comp))
        else:
            m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e4, index=('Liq', comp))
    
    # Calculate scaling factors
    calculate_scaling_factors(m)
    
    # Get solver
    solver = get_solver()
    solver.options['max_iter'] = 200
    
    # Check initial DOF
    print(f"\nInitial degrees of freedom: {degrees_of_freedom(m)}")
    
    # Add recovery constraints for each stage to achieve 0 DOF
    # Use conservative recovery targets to improve feasibility
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        
        # Use a more conservative recovery target
        target_recovery = stage_data.get('stage_recovery', 0.5)
        # Limit recovery to reasonable values
        target_recovery = min(target_recovery, 0.6 if i == 1 else 0.5)
        
        # Add constraint for water recovery
        setattr(
            m.fs, f"recovery_constraint_stage{i}",
            Constraint(
                expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] == target_recovery
            )
        )
    
    print(f"Degrees of freedom after recovery constraints: {degrees_of_freedom(m)}")
    
    # Solve
    print("\nSolving model to determine required pump pressures...")
    results = solver.solve(m, tee=False)
    
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("\nSolution found!")
        print("\nCalculated Pump Pressures:")
        for i in range(1, config_data['stage_count'] + 1):
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            
            feed_pressure = value(pump.outlet.pressure[0])
            inlet_pressure = value(pump.inlet.pressure[0])
            power_kw = value(pump.work_mechanical[0]) / 1000
            recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
            
            print(f"\n  Stage {i}:")
            print(f"    Pump inlet pressure: {inlet_pressure/1e5:.1f} bar")
            print(f"    Required feed pressure: {feed_pressure/1e5:.1f} bar")
            print(f"    Pressure boost: {(feed_pressure-inlet_pressure)/1e5:.1f} bar")
            print(f"    Pump power: {power_kw:.1f} kW")
            print(f"    Water recovery: {recovery:.1%}")
    else:
        print(f"\nSolver failed: {results.solver.termination_condition}")
    
    if optimize_pumps and results.solver.termination_condition == TerminationCondition.optimal:
        print("\nOptimizing pump pressures...")
        
        # Remove recovery constraints to allow optimization
        for i in range(1, config_data['stage_count'] + 1):
            constraint = getattr(m.fs, f"recovery_constraint_stage{i}")
            constraint.deactivate()
        
        # Add objective to minimize total power
        m.fs.total_power = Var(initialize=100, bounds=(0, 10000))
        m.fs.power_constraint = Constraint(
            expr=m.fs.total_power == sum(
                getattr(m.fs, f"pump{i}").work_mechanical[0] / 1000
                for i in range(1, config_data['stage_count'] + 1)
            )
        )
        
        # Add constraints to maintain minimum recovery
        for i in range(1, config_data['stage_count'] + 1):
            ro = getattr(m.fs, f"ro_stage{i}")
            stage_data = config_data['stages'][i-1]
            min_recovery = stage_data.get('stage_recovery', 0.5) * 0.95  # 95% of target
            
            setattr(
                m.fs, f"min_recovery_constraint_stage{i}",
                Constraint(
                    expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] >= min_recovery
                )
            )
        
        m.fs.objective = Objective(expr=m.fs.total_power, sense=minimize)
        
        # Re-solve
        results = solver.solve(m, tee=False)
    
    return results

def solve_with_scaling_simple(m, config_data, optimize_pumps=False):
    """
    Solve simplified model with appropriate scaling.
    Pump pressures are calculated to meet target recoveries.
    """
    # Apply scaling factors for mass flow basis
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e2, index=('Liq', 'H2O'))
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e4, index=('Liq', 'Na+'))
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e4, index=('Liq', 'Cl-'))
    
    # Calculate scaling factors
    calculate_scaling_factors(m)
    
    # Get solver
    solver = get_solver()
    solver.options['max_iter'] = 200
    
    # Check initial DOF
    print(f"\nInitial degrees of freedom: {degrees_of_freedom(m)}")
    
    # Add recovery constraints for each stage to achieve 0 DOF
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        target_recovery = stage_data.get('stage_recovery', 0.5)
        
        # Add constraint for water recovery
        setattr(
            m.fs, f"recovery_constraint_stage{i}",
            Constraint(
                expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] == target_recovery
            )
        )
    
    print(f"Degrees of freedom after recovery constraints: {degrees_of_freedom(m)}")
    
    # Solve
    print("\nSolving model to determine required pump pressures...")
    results = solver.solve(m, tee=True)
    
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("\nSolution found!")
        print("\nCalculated Pump Pressures:")
        for i in range(1, config_data['stage_count'] + 1):
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            
            feed_pressure = value(pump.outlet.pressure[0])
            inlet_pressure = value(pump.inlet.pressure[0])
            power_kw = value(pump.work_mechanical[0]) / 1000
            recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
            
            print(f"\n  Stage {i}:")
            print(f"    Pump inlet pressure: {inlet_pressure/1e5:.1f} bar")
            print(f"    Required feed pressure: {feed_pressure/1e5:.1f} bar")
            print(f"    Pressure boost: {(feed_pressure-inlet_pressure)/1e5:.1f} bar")
            print(f"    Pump power: {power_kw:.1f} kW")
            print(f"    Water recovery: {recovery:.1%}")
    
    if optimize_pumps and results.solver.termination_condition == TerminationCondition.optimal:
        print("\nOptimizing pump pressures...")
        
        # Remove recovery constraints to allow optimization
        for i in range(1, config_data['stage_count'] + 1):
            constraint = getattr(m.fs, f"recovery_constraint_stage{i}")
            constraint.deactivate()
        
        # Add objective to minimize total power
        m.fs.total_power = Var(initialize=100, bounds=(0, 10000))
        m.fs.power_constraint = Constraint(
            expr=m.fs.total_power == sum(
                getattr(m.fs, f"pump{i}").work_mechanical[0] / 1000
                for i in range(1, config_data['stage_count'] + 1)
            )
        )
        
        # Add constraints to maintain minimum recovery
        for i in range(1, config_data['stage_count'] + 1):
            ro = getattr(m.fs, f"ro_stage{i}")
            stage_data = config_data['stages'][i-1]
            min_recovery = stage_data.get('stage_recovery', 0.5) * 0.95  # 95% of target
            
            setattr(
                m.fs, f"min_recovery_constraint_stage{i}",
                Constraint(
                    expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] >= min_recovery
                )
            )
        
        m.fs.objective = Objective(expr=m.fs.total_power, sense=minimize)
        
        # Re-solve
        results = solver.solve(m, tee=True)
    
    return results

In [ ]:
def predict_scaling_potential(m, stage_num):
    """
    Predict scaling potential using concentrate composition.
    """
    # For now, return a simplified scaling assessment
    # In production, this would use a proper scaling prediction module
    
    ro = getattr(m.fs, f"ro_stage{stage_num}")
    conc_state = ro.retentate
    
    # Extract ion concentrations in mg/L
    conc_ions_mg_l = {}
    total_flow_kg_s = 0
    
    for comp in m.fs.properties.component_list:
        if comp != "H2O":
            # Get mass flow directly (already in mass basis)
            comp_flow_kg_s = value(conc_state.flow_mass_phase_comp[0, 'Liq', comp])
            conc_ions_mg_l[comp] = comp_flow_kg_s  # Will convert to mg/L later
        else:
            water_flow_kg_s = value(conc_state.flow_mass_phase_comp[0, 'Liq', 'H2O'])
            total_flow_kg_s += water_flow_kg_s
    
    # Convert to mg/L
    total_flow_m3_s = total_flow_kg_s / 1000  # Approximate density 1000 kg/m³
    if total_flow_m3_s > 0:
        for ion in conc_ions_mg_l:
            # kg/s / (m³/s) * 1e6 mg/kg = mg/L
            conc_ions_mg_l[ion] = conc_ions_mg_l[ion] / total_flow_m3_s * 1e6
    
    # Simple scaling assessment based on common scaling compounds
    scaling_results = {}
    
    # Calcium carbonate scaling (simplified)
    if 'Ca2+' in conc_ions_mg_l and 'HCO3-' in conc_ions_mg_l:
        ca_mol_l = conc_ions_mg_l.get('Ca2+', 0) / 40080  # mg/L to mol/L
        hco3_mol_l = conc_ions_mg_l.get('HCO3-', 0) / 61020  # mg/L to mol/L
        # Very simplified Langelier Saturation Index approximation
        lsi_approx = 0.5 * (ca_mol_l * hco3_mol_l * 1e6) - 1.0
        scaling_results['CaCO3'] = {
            'saturation_index': lsi_approx,
            'scaling_tendency': 'High' if lsi_approx > 0.5 else 'Medium' if lsi_approx > 0 else 'Low'
        }
    
    # Calcium sulfate scaling
    if 'Ca2+' in conc_ions_mg_l and 'SO4-2' in conc_ions_mg_l:
        ca_mg_l = conc_ions_mg_l.get('Ca2+', 0)
        so4_mg_l = conc_ions_mg_l.get('SO4-2', 0)
        # Simple check against typical solubility
        if (ca_mg_l * so4_mg_l) > 500000:  # Simplified threshold
            scaling_results['CaSO4'] = {
                'saturation_index': 1.0,
                'scaling_tendency': 'High'
            }
        else:
            scaling_results['CaSO4'] = {
                'saturation_index': -0.5,
                'scaling_tendency': 'Low'
            }
    
    # Antiscalant recommendation (simplified)
    antiscalant_rec = {
        'antiscalant_type': 'None required',
        'dosage_ppm': 0,
        'primary_concern': 'None'
    }
    
    # Check if any scaling risk exists
    high_risk_scales = [k for k, v in scaling_results.items() 
                        if v.get('scaling_tendency') == 'High']
    
    if high_risk_scales:
        if 'CaCO3' in high_risk_scales:
            antiscalant_rec = {
                'antiscalant_type': 'Phosphonate-based',
                'dosage_ppm': 3.0,
                'primary_concern': 'Calcium carbonate',
                'specific_products': ['FLOCON 260', 'Vitec 3000']
            }
        elif 'CaSO4' in high_risk_scales:
            antiscalant_rec = {
                'antiscalant_type': 'Polyacrylate-based',
                'dosage_ppm': 2.5,
                'primary_concern': 'Calcium sulfate',
                'specific_products': ['FLOCON 100', 'Vitec 2000']
            }
    
    return scaling_results, antiscalant_rec

## Extract Results

In [ ]:
def extract_results_mcas(m, config_data):
    """
    Extract results including detailed ion concentrations.
    """
    results = {
        "status": "success",
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {},
        "ion_analysis": {}
    }
    
    # Get feed flow (mass basis)
    feed_vol_flow = sum(
        value(m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', comp])  # kg/s
        for comp in m.fs.properties.component_list
    ) / 1000  # m³/s assuming density ~1000
    
    # Total permeate flow
    total_perm_flow = sum(
        sum(
            value(getattr(m.fs, f"stage_product{i}").inlet.flow_mass_phase_comp[0, 'Liq', comp])
            for comp in m.fs.properties.component_list
        ) / 1000
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    results["performance"]["total_recovery"] = total_perm_flow / feed_vol_flow
    
    # Energy consumption
    total_power_kw = sum(
        value(getattr(m.fs, f"pump{i}").work_mechanical[0]) / 1000
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    results["economics"]["total_power_kw"] = total_power_kw
    results["economics"]["specific_energy_kwh_m3"] = (
        total_power_kw / (total_perm_flow * 3600)
    )
    
    # Stage results with ion details
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Calculate flows and TDS
        perm_tds = 0
        perm_ions = {}
        
        for comp in m.fs.properties.component_list:
            if comp != "H2O":
                perm_kg_s = value(ro.permeate.flow_mass_phase_comp[0, 'Liq', comp])
                perm_mg_s = perm_kg_s * 1e6  # kg/s to mg/s
                perm_flow_m3_s = value(ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O']) / 1000
                perm_conc_mg_l = perm_mg_s / (perm_flow_m3_s * 1000) if perm_flow_m3_s > 0 else 0
                perm_ions[comp] = perm_conc_mg_l
                perm_tds += perm_conc_mg_l
        
        # Similar for concentrate
        conc_tds = 0
        conc_ions = {}
        
        for comp in m.fs.properties.component_list:
            if comp != "H2O":
                conc_kg_s = value(ro.retentate.flow_mass_phase_comp[0, 'Liq', comp])
                conc_mg_s = conc_kg_s * 1e6  # kg/s to mg/s
                conc_flow_m3_s = value(ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O']) / 1000
                conc_conc_mg_l = conc_mg_s / (conc_flow_m3_s * 1000) if conc_flow_m3_s > 0 else 0
                conc_ions[comp] = conc_conc_mg_l
                conc_tds += conc_conc_mg_l
        
        # Predict scaling for this stage
        scaling_results, antiscalant_rec = predict_scaling_potential(m, i)
        
        stage_result = {
            "stage_number": i,
            "feed_pressure_bar": value(ro.inlet.pressure[0]) / 1e5,
            "pressure_drop_bar": -value(ro.deltaP[0]) / 1e5,  # Convert to positive
            "permeate_flow_m3h": sum(
                value(ro.permeate.flow_mass_phase_comp[0, 'Liq', comp])
                for comp in m.fs.properties.component_list
            ) / 1000 * 3600,
            "permeate_tds_ppm": perm_tds,
            "permeate_ions_mg_l": perm_ions,
            "concentrate_flow_m3h": sum(
                value(ro.retentate.flow_mass_phase_comp[0, 'Liq', comp])
                for comp in m.fs.properties.component_list
            ) / 1000 * 3600,
            "concentrate_tds_ppm": conc_tds,
            "concentrate_ions_mg_l": conc_ions,
            "pump_power_kw": value(getattr(m.fs, f"pump{i}").work_mechanical[0]) / 1000,
            "membrane_area_m2": value(ro.area),
            "scaling_indices": {
                mineral: data["saturation_index"] 
                for mineral, data in scaling_results.items() 
                if "saturation_index" in data
            },
            "scaling_risks": {
                mineral: data["scaling_tendency"] 
                for mineral, data in scaling_results.items() 
                if "scaling_tendency" in data
            },
            "antiscalant_recommendation": antiscalant_rec
        }
        
        results["stage_results"].append(stage_result)
    
    # Overall ion rejection
    overall_rejection = {}
    for comp in m.fs.properties.component_list:
        if comp != "H2O":
            feed_kg_s = value(m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', comp])
            perm_kg_s = sum(
                value(getattr(m.fs, f"stage_product{i}").inlet.flow_mass_phase_comp[0, 'Liq', comp])
                for i in range(1, config_data['stage_count'] + 1)
            )
            if feed_kg_s > 0:
                overall_rejection[comp] = 1 - (perm_kg_s / feed_kg_s)
    
    results["ion_analysis"]["overall_rejection"] = overall_rejection
    
    # Mass balance
    results["mass_balance"]["feed_flow_m3h"] = feed_vol_flow * 3600
    results["mass_balance"]["total_permeate_m3h"] = total_perm_flow * 3600
    results["mass_balance"]["final_concentrate_m3h"] = sum(
        value(m.fs.concentrate_product.inlet.flow_mass_phase_comp[0, 'Liq', comp])
        for comp in m.fs.properties.component_list
    ) / 1000 * 3600
    
    balance_error = abs(
        results["mass_balance"]["feed_flow_m3h"] - 
        results["mass_balance"]["total_permeate_m3h"] - 
        results["mass_balance"]["final_concentrate_m3h"]
    )
    results["mass_balance"]["error_m3h"] = balance_error
    results["mass_balance"]["error_percent"] = (
        balance_error / results["mass_balance"]["feed_flow_m3h"] * 100
    )
    
    return results

## Run Simulation

In [ ]:
# Build and solve model
try:
    print("Building simplified RO model with MCAS property package...")
    m = build_ro_model_mcas_simple(configuration, mcas_config, feed_temperature_c, membrane_type)
    
    print(f"\nInitializing model...")
    initialize_model_simple(m, configuration)
    
    print("\nSolving model...")
    solve_results = solve_with_scaling_simple(m, configuration, optimize_pumps)
    
    if solve_results.solver.termination_condition == TerminationCondition.optimal:
        print("\nExtracting results...")
        results = extract_results_mcas(m, configuration)
        results["message"] = "Simulation with MCAS completed successfully"
        results["property_package"] = "MCAS"
        results["model_type"] = "simplified_fixed_pressure_drop"
        results["components_modeled"] = ['H2O'] + mcas_config['solute_list']
        results["initialization_strategy"] = initialization_strategy
        
        # Add optimization results if performed
        if optimize_pumps and hasattr(m.fs, 'total_power'):
            results["optimization"] = {
                "performed": True,
                "minimized_power_kw": value(m.fs.total_power),
                "optimized_pressures_bar": {
                    f"stage_{i}": value(getattr(m.fs, f"pump{i}").outlet.pressure[0])/1e5
                    for i in range(1, configuration['stage_count'] + 1)
                }
            }
    else:
        results = {
            "status": "error",
            "message": f"Solver failed: {solve_results.solver.termination_condition}",
            "performance": {},
            "economics": {},
            "stage_results": [],
            "mass_balance": {},
            "ion_analysis": {}
        }
        
except Exception as e:
    import traceback
    results = {
        "status": "error",
        "message": f"Simulation error: {str(e)}",
        "traceback": traceback.format_exc(),
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {},
        "ion_analysis": {}
    }

print("\nSimulation complete.")

## Display Results

In [ ]:
# Display results summary
import json
print("\n" + "="*60)
print("SIMULATION RESULTS - SIMPLIFIED MCAS MODEL")
print("="*60)

if results.get("status") == "success":
    print(f"\nOverall Performance:")
    print(f"  Total Recovery: {results['performance']['total_recovery']:.1%}")
    print(f"  Specific Energy: {results['economics']['specific_energy_kwh_m3']:.2f} kWh/m³")
    print(f"  Total Power: {results['economics']['total_power_kw']:.1f} kW")
    
    print(f"\nIon Rejection:")
    for ion, rejection in results['ion_analysis']['overall_rejection'].items():
        print(f"  {ion:8s}: {rejection:.1%}")
    
    print(f"\nStage Results:")
    for stage in results['stage_results']:
        print(f"\n  Stage {stage['stage_number']}:")
        print(f"    Pressure: {stage['feed_pressure_bar']:.1f} bar (ΔP = {stage['pressure_drop_bar']:.1f} bar)")
        print(f"    Permeate: {stage['permeate_flow_m3h']:.1f} m³/h @ {stage['permeate_tds_ppm']:.0f} ppm")
        print(f"    Concentrate: {stage['concentrate_flow_m3h']:.1f} m³/h @ {stage['concentrate_tds_ppm']:.0f} ppm")
        print(f"    Membrane area: {stage['membrane_area_m2']:.0f} m²")
        
        # Show scaling risks
        if stage.get('scaling_indices'):
            print(f"    Scaling Risk Assessment:")
            for mineral, si in sorted(stage['scaling_indices'].items()):
                if si > -0.5:  # Only show minerals near or above saturation
                    risk = stage['scaling_risks'].get(mineral, '')
                    print(f"      {mineral:12s}: SI = {si:+.2f} - {risk}")
        
        # Show antiscalant recommendation if scaling risk exists
        if stage.get('antiscalant_recommendation'):
            rec = stage['antiscalant_recommendation']
            if rec['antiscalant_type'] != "None required":
                print(f"    Antiscalant Recommendation:")
                print(f"      Primary concern: {rec['primary_concern']}")
                print(f"      Type: {rec['antiscalant_type']}")
                print(f"      Dosage: {rec['dosage_ppm']:.1f} ppm")
                if rec.get('specific_products'):
                    print(f"      Products: {', '.join(rec['specific_products'][:2])}")

# Full results as JSON
print("\n" + "="*60)
print("FULL RESULTS (JSON):")
print("="*60)
print(json.dumps(results, indent=2, default=str))

In [ ]:
# Results cell - tagged for papermill to extract
results